In [ ]:
# this lets us use the figures interactively
%matplotlib inline 

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import cartopy.crs  as ccrs
from shapely.geometry import Point, LineString, Polygon
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cartopy.feature import ShapelyFeature




# ---------------------------------------------------------------------------------------------------------------------
Counties = gpd.read_file('data_files/Counties.shp')#  to load the ward and county data and complete the main part of the analysis.
NIWards = gpd.read_file('data_files/NI_Wards.shp')

# load the necessary data here and transform to a UTM projection
#Counties_itm = Counties.to_crs(epsg=2157) # transform to Irish Transverse Mercator using EPSG2157
#NIWards_itm = Counties.to_crs(epsg=2157) # transform to Irish Transverse Mercator using EPSG2157

for ind, row in NIWards.iterrows(): # iterate over each row in the GeoDataFrame
    NIWards.loc[ind, 'Area'] = row['geometry'].area # assign area to new field 

# your analysis goes here...
join = gpd.sjoin(Counties, NIWards, how='inner', lsuffix='left', rsuffix='right') # perform the spatial join of counties and wards
joinedData = join.groupby(['CountyName'])['Population'].sum() # assign summary data to variable to be printed
print(joinedData)





# ---------------------------------------------------------------------------------------------------------------------

# below here, you may need to modify the script somewhat to create your map.
# create a crs using ccrs.UTM() that corresponds to our CRS
myCRS = ccrs.UTM(29)
# create a figure of size 10x10 (representing the page size in inches


fig, ax = plt.subplots(1, 1, figsize=(10, 10), subplot_kw=dict(projection=myCRS))



# add gridlines below
gridlines = ax.gridlines(draw_labels=True,
                         xlocs=[-8, -7.5, -7, -6.5, -6, -5.5],
                         ylocs=[54, 54.5, 55, 55.5],
                         linewidth = 0.5, color = 'gray', alpha = 0.5, linestyle='--')

gridlines.right_labels = False
gridlines.bottom_labels = False
gridlines.left_labels = True
gridlines.top_labels = True

# to make a nice colorbar that stays in line with our map, use these lines:
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1, axes_class=plt.Axes)


# plot the ward data into our axis, using
ward_plot = NIWards.plot(column='Population', ax=ax, vmin=1000, vmax=8000, cmap='viridis',
                       legend=True, cax=cax, legend_kwds={'label': 'Resident Population'})


county_outlines = ShapelyFeature(Counties['geometry'], myCRS, edgecolor='r', facecolor='none')


ax.add_feature(county_outlines)
county_handles = [mpatches.Rectangle((0, 0), 1, 1, facecolor='none', edgecolor='r')]

ax.legend(county_handles, ['County Boundaries'], fontsize=12, loc='upper left', framealpha=1)

# save the figure
fig.savefig('sample_map.png', dpi=300, bbox_inches='tight')




In [ ]:
NIWards.head()

In [ ]:
print(NIWards.shape)
print(join.shape)

In [ ]:
join.head()

In [ ]:
join[join['Population']==join['Population'].max()]

In [ ]:
join[join['Population']==join['Population'].min()]

In [ ]:
print(join.columns)



In [ ]:
print(type(join))
a =join.groupby(['CountyName', 'Ward Code'])['Ward Code'].count()


In [ ]:
a.shape
print(a)




In [ ]:
join.rename(columns={"WardCode":"WardCode"},inplace=True)
join.columns



In [ ]:
join.groupby(['WardCode', 'CountyName'])['Population'].sum()

In [ ]:
join_total = join['Population'].sum()
wards_total = NIWards['Population'].sum()
print(join.groupby(['CountyName', 'Ward'])['Population'].sum())
print('Population in wards item is: ', wards_total)
print('Population from joined table is: ', join_total)
popDupl= join_total-wards_total 
print('Population in duplicated wards = ', popDupl)